<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#STAR" data-toc-modified-id="STAR-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>STAR</a></span><ul class="toc-item"><li><span><a href="#Situation" data-toc-modified-id="Situation-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Situation</a></span></li><li><span><a href="#Threat" data-toc-modified-id="Threat-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Threat</a></span></li><li><span><a href="#Action" data-toc-modified-id="Action-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Action</a></span></li><li><span><a href="#Result" data-toc-modified-id="Result-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Result</a></span></li></ul></li><li><span><a href="#Workflow-2:-Don't-Drop-Labels" data-toc-modified-id="Workflow-2:-Don't-Drop-Labels-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Workflow 2: Don't Drop Labels</a></span></li><li><span><a href="#Original-Workflow" data-toc-modified-id="Original-Workflow-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Original Workflow</a></span></li></ul></div>

# Intro

# STAR

## Situation

---

* NA uses DRR summary reports to post revenue in LS
* I use report packet to review for DRR

---

## Threat

---

* **Pro:** clearly provide total sum amounts for audit to post directly

* **Cons:**
    * "Food" = "Food" + "Other"
    * Discounts are ambiguous
        * FBC vs. Assoc vs. others
    * *LOTS* of unused rows/labels create visual confusion
    * Not always clear to which XAC to post revenue/tax/tips
    
---
    

## Action

---

**End Goal:**

|Sub-Dept | Rain 903 | IRD  | CC | Bar|
| ----- | ----- | ----- | ----- | ----- |
|Food | 1.00 | 1.00 | 1.00 | 1.00 |
|Liquor | 2.00 | 2.00 | 2.00 | 2.00 
|Tax | 3.00 | 3.00 | 3.00 | 3.00 
|Gratuity | 4.00 | 4.00 | 4.00 | 4.00 

---

## Result

In [ ]:
## --- Importing Packages --- ##

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import bmcds.eda as eda

pd.set_option('display.max_rows', None)

In [ ]:
## --- Specify file path --- ##
file_path = './data/02_25_2022_pc.xls'

In [ ]:
df = pd.read_excel(file_path, header = None, skiprows = 3)
df

In [ ]:
df.columns = ['Label', 'Bar', 'drop1', 'Dining Room', 'drop2', 'Room Service', 'drop3', 'Starbucks', 'drop4', 'Page Total']
df

In [ ]:
df = df.drop(columns = ['drop1', 'drop2', 'drop3', 'drop4', 'Page Total'])
df

In [ ]:
df.isna().sum(axis=1).value_counts()

In [ ]:
df_cleaned = df[df.isna().sum(axis=1) ==0]
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned

# Workflow 2: Don't Drop Labels

In [ ]:
df_no_drp = df_cleaned.copy()
df_no_drp

In [ ]:
df_no_drp_idx = df_no_drp[~df_no_drp.loc[:, "Label"].isin([' '])]
df_no_drp_idx

In [ ]:
df_no_drp_idx = df_no_drp_idx.rename(df_no_drp_idx["Label"])
df_no_drp_idx = df_no_drp_idx.drop(columns = "Label")
df_no_drp_idx

In [ ]:
df_no_drp_idx.dtypes

In [ ]:
for col in df_no_drp_idx.columns:
    df_no_drp_idx[col] = pd.to_numeric(df_no_drp_idx[col], downcast = 'float')

In [ ]:
df_no_drp_idx.dtypes

In [ ]:
df_no_drp_idx

# Original Workflow

In [ ]:
df_cleaned.loc[:, "Label"].unique()

In [ ]:
drop_labels = ['Sales Subtotal', 'Total before tax', 'None', 'None', 
               'Tax Subtotal','Receipt Rounding','Total of check', ' ', 'Unadjusted receipts',
               'PaidIn - AR', 'PaidIn - Debitek','PaidIn - Gift Certificates',
               'No of Covers', 'No of Checks', 'Avg cover', 'Avg check',
               'Total Paid Outs', 'Server bank', 'Total tips',
               'Server credit card fees', 'Net tips', 'Net Cash',
               'Other receipts turned in', 'Server Drops', 'Turned in',
               'Restaurant credit card fees', 'Estimated deposit',
               'Beginning NRS Total', 'Total Receipts', 'Discounts',
               'Paid Ins','Gross Total', 'NRS Total']
drop_labels

In [ ]:
df_cln_idx =df_cleaned[~df_cleaned.loc[:, "Label"].isin(drop_labels)]
df_cln_idx

In [ ]:
df_new_idx = df_cln_idx.copy()

In [ ]:
df_new_idx = df_new_idx.rename(df_cleaned["Label"])
df_new_idx = df_new_idx.drop(columns = "Label")
df_new_idx

In [ ]:
df_new_idx.dtypes

In [ ]:
for col in df_new_idx.columns:
    df_new_idx[col] = pd.to_numeric(df_new_idx[col], downcast = 'float')

In [ ]:
df_new_idx.dtypes

In [ ]:
df_new_idx.T.describe()

In [ ]:
df_new_idx['Total'] = df_new_idx.sum(axis=1)
df_new_idx

In [ ]:
df_compressed = df_new_idx[df_new_idx['Total'] >0]
df_compressed

In [ ]:
df_compressed['Bar']['Food']

In [ ]:
for col in df_compressed.columns[:-1]:
    print(f"Post to {col} food: {df_compressed[col]['Food']+df_compressed[col]['Other']:.2f}")

In [ ]:
dept_list = ["Rain 903", "In Room Dining", "Coffee Corner", "Lobby Bar"]
sd_list = ["Food", "Liquor", "Tax", "Gratuity"]

for dept in dept_list:
    for sd in sd_list:
        print(dept+" - "+sd)

In [ ]:
df_compressed

In [ ]:
df